# StyleGAN2: unsupervised discovery of latent directions

Reference: https://github.com/woctezuma/steam-stylegan2


## Papers

[1] [Voynov, Andrey, et al. *Unsupervised Discovery of Interpretable Directions in Latent Space*. ICML 2020.][GanLatentDiscovery-paper] **([code][GanLatentDiscovery])**

[GanLatentDiscovery-paper]: <https://arxiv.org/abs/2002.03754>
[GanLatentDiscovery]: <https://github.com/anvoynov/GanLatentDiscovery>

[2] [Härkönen, Erik, et al. *GANSpace: Discovering Interpretable GAN Controls*. arXiv 2020.][ganspace-paper] **([code][ganspace])**

[ganspace-paper]: <https://arxiv.org/abs/2004.02546>
[ganspace]: <https://github.com/harskish/ganspace>

[3] [Shen, Yujun, et al. *Closed-Form Factorization of Latent Semantics in GANs*. arXiv 2020.][closed-form-paper] **([code][closed-form])**

[closed-form-paper]: <https://arxiv.org/abs/2007.06600>
[closed-form]: <https://github.com/rosinality/stylegan2-pytorch#closed-form-factorization-httpsarxivorgabs200706600>

### Issues

NB: code [1] cannot be run in a Google Colab session, due to [a memory issue](https://github.com/anvoynov/GANLatentDiscovery/issues/13):
```
RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 14.73 
GiB total capacity; 13.20 GiB already allocated; 499.88 MiB free; 13.36 GiB 
reserved in total by PyTorch)
```

NB: code [2] [can be run](https://github.com/harskish/ganspace/issues/4) with `config-e` of StyleGAN2, thanks to [my `steam` fork](https://github.com/woctezuma/ganspace/tree/steam) which includes the most obvious fixes.

If you try to follow [this notebook](https://colab.research.google.com/drive/1g-ShMzkRWDMHPyjom_p-5kqkn2f-GwBi) with my fork, you will encounter this error, but only if you forget to specify a layer to explore:
```
Traceback (most recent call last):
  File "visualize.py", line 152, in <module>
    inst = get_instrumented_model(args.model, args.output_class, layer_key, device, use_w=args.use_w)
  File "/usr/lib/python3.6/functools.py", line 807, in wrapper
    return dispatch(args[0].__class__)(*args, **kw)
  File "/content/ganspace/models/wrappers.py", line 726, in get_instrumented_model
    latent_shape = model.get_latent_shape()
  File "/content/ganspace/netdissect/modelconfig.py", line 98, in create_instrumented_model
    model.retain_layers(args.layers)
  File "/content/ganspace/netdissect/nethook.py", line 63, in retain_layers
    self.add_hooks(layernames)
  File "/content/ganspace/netdissect/nethook.py", line 164, in add_hooks
    raise ValueError('Layer %s not found in model' % name)
ValueError: Layer g_mapping not found in model
```

## Requirements

### Check the GPU

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-eb898157-ff44-253a-adc1-105e488192ac)


### Switch to Tensorflow 1.x

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


### Mount Google Drive

In [ ]:
%pip install Google-Colab-Transfer

In [ ]:
import colab_transfer

colab_transfer.mount_google_drive()

Mounted at /content/drive/


## Load the last checkpoint from Google Drive

In [ ]:
debug_with_ffhq = True
use_config_e = False

In [ ]:
if debug_with_ffhq:

  if use_config_e:
    tensorflow_model_name = 'stylegan2-ffhq-config-e.pkl'
  else:
    tensorflow_model_name = 'stylegan2-ffhq-config-f.pkl'
    
else:
  tensorflow_model_name = 'network-snapshot-005000.pkl'

pytorch_model_name = tensorflow_model_name.replace('.pkl', '.pt')

### Import the Tensorflow checkpoint from Google Drive

In [ ]:
%cd /content/

/content


In [ ]:
if debug_with_ffhq:
  # Reference: https://github.com/NVlabs/stylegan2/blob/master/pretrained_networks.py  
  !wget http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/$tensorflow_model_name
else:
  colab_transfer.copy_file(file_name=tensorflow_model_name,
                          source=colab_transfer.get_path_to_home_of_google_drive() + 'checkpoints/',
                          destination=colab_transfer.get_path_to_home_of_local_machine())

--2020-09-20 14:38:54--  http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-e.pkl
Resolving d36zk2xti64re0.cloudfront.net (d36zk2xti64re0.cloudfront.net)... 13.224.10.24, 13.224.10.81, 13.224.10.190, ...
Connecting to d36zk2xti64re0.cloudfront.net (d36zk2xti64re0.cloudfront.net)|13.224.10.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 318021648 (303M) [application/x-www-form-urlencoded]
Saving to: ‘stylegan2-ffhq-config-e.pkl.1’

stylegan2-ffhq-conf 100%[===================>] 303.29M   300MB/s    in 1.0s    

2020-09-20 14:38:56 (300 MB/s) - ‘stylegan2-ffhq-config-e.pkl.1’ saved [318021648/318021648]



### Convert the checkpoint from TensorFlow to PyTorch

Reference: https://github.com/rosinality/stylegan2-pytorch

#### Clone StyleGAN2 (TensorFlow)

In [ ]:
%cd /content/
!git clone https://github.com/NVlabs/stylegan2

/content
Cloning into 'stylegan2'...
remote: Enumerating objects: 93, done.
remote: Total 93 (delta 0), reused 0 (delta 0), pack-reused 93
Unpacking objects: 100% (93/93), done.


#### Clone StyleGAN2 (PyTorch)

In [ ]:
%pip install ninja

     |████████████████████████████████| 112kB 6.6MB/s 


In [ ]:
# %cd /content/
# !git clone https://github.com/rosinality/stylegan2-pytorch.git

In [ ]:
%cd /content/
!git clone https://github.com/woctezuma/stylegan2-pytorch.git

/content
Cloning into 'stylegan2-pytorch'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 379 (delta 40), reused 44 (delta 26), pack-reused 312
Receiving objects: 100% (379/379), 122.50 MiB | 42.94 MiB/s, done.
Resolving deltas: 100% (189/189), done.


In [ ]:
%cd /content/stylegan2-pytorch/
!git pull
!git checkout quality-of-life-merge

/content/stylegan2-pytorch
Already up to date.
Branch 'quality-of-life-merge' set up to track remote branch 'quality-of-life-merge' from 'origin'.
Switched to a new branch 'quality-of-life-merge'


#### Convert

In [ ]:
%cd /content/stylegan2-pytorch/

/content/stylegan2-pytorch


For `config-e`, set `--channel_multiplier` to 1.

Reference: https://github.com/rosinality/stylegan2-pytorch/issues/52

In [ ]:
if 'config-f' in tensorflow_model_name:
  channel_multiplier = 2
else:
  channel_multiplier = 1

In [ ]:
if debug_with_ffhq:
  !python convert_weight.py \
  --repo /content/stylegan2/ \
  --channel_multiplier $channel_multiplier \
  /content/$tensorflow_model_name

Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
2020-09-20 14:39:30.427629: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 16777216 exceeds 10% of system memory.
tensor(6.5267e-05, device='cuda:0')


#### Export the converted checkpoint to Google Drive

In [ ]:
# if not debug_with_ffhq:
#   colab_transfer.copy_file(file_name=pytorch_model_name,
#                           source=colab_transfer.get_path_to_home_of_local_machine() + 'stylegan2-pytorch/',
#                           destination=colab_transfer.get_path_to_home_of_google_drive() + 'checkpoints/pytorch/')

#### Import the PyTorch checkpoint from Google Drive

In [ ]:
if not debug_with_ffhq:
  colab_transfer.copy_file(file_name=pytorch_model_name,
                          source=colab_transfer.get_path_to_home_of_google_drive() + 'checkpoints/pytorch/',
                          destination=colab_transfer.get_path_to_home_of_local_machine() + 'stylegan2-pytorch/')

## Check the checkpoints

In [ ]:
# TensorFlow
!du -sh /content/$tensorflow_model_name

# PyTorch
!du -sh /content/stylegan2-pytorch/$pytorch_model_name

304M	/content/stylegan2-ffhq-config-e.pkl
106M	/content/stylegan2-pytorch/stylegan2-ffhq-config-e.pt


## Show the average image

Reference: https://gist.github.com/woctezuma/139cedb92a94c5ef2675cc9f06851b31

In [ ]:
%cd /content/

/content


In [ ]:
from pathlib import Path

ffhq_model_names = [
                  'stylegan2-ffhq-config-e.pkl',
                  'stylegan2-ffhq-config-f.pkl',
                  ]

for model_name in ffhq_model_names:
  if not Path('/content/' + model_name).exists():
    url = 'http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/' + model_name
    !wget $url

--2020-09-21 13:53:46--  http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-e.pkl
Resolving d36zk2xti64re0.cloudfront.net (d36zk2xti64re0.cloudfront.net)... 13.224.10.190, 13.224.10.184, 13.224.10.81, ...
Connecting to d36zk2xti64re0.cloudfront.net (d36zk2xti64re0.cloudfront.net)|13.224.10.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 318021648 (303M) [application/x-www-form-urlencoded]
Saving to: ‘stylegan2-ffhq-config-e.pkl’

stylegan2-ffhq-conf 100%[===================>] 303.29M  16.2MB/s    in 20s     

2020-09-21 13:54:07 (15.2 MB/s) - ‘stylegan2-ffhq-config-e.pkl’ saved [318021648/318021648]

--2020-09-21 13:54:07--  http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl
Resolving d36zk2xti64re0.cloudfront.net (d36zk2xti64re0.cloudfront.net)... 13.224.10.81, 13.224.10.190, 13.224.10.24, ...
Connecting to d36zk2xti64re0.cloudfront.net (d36zk2xti64re0.cloudfront.net)|13.224.10.81|:80... connected

In [ ]:
model_names = [
                'network-snapshot-001014.pkl',
                'network-snapshot-002003.pkl',
                'network-snapshot-003001.pkl',
                'network-snapshot-004006.pkl',
                'network-snapshot-005000.pkl',
               ]

for model_name in model_names:
  colab_transfer.copy_file(file_name=model_name,
                          source=colab_transfer.get_path_to_home_of_google_drive() + 'checkpoints/',
                          destination=colab_transfer.get_path_to_home_of_local_machine())

Copying /content/drive/My Drive/checkpoints/network-snapshot-001014.pkl to /content/network-snapshot-001014.pkl
Copying /content/drive/My Drive/checkpoints/network-snapshot-002003.pkl to /content/network-snapshot-002003.pkl
Copying /content/drive/My Drive/checkpoints/network-snapshot-003001.pkl to /content/network-snapshot-003001.pkl
Copying /content/drive/My Drive/checkpoints/network-snapshot-004006.pkl to /content/network-snapshot-004006.pkl
Copying /content/drive/My Drive/checkpoints/network-snapshot-005000.pkl to /content/network-snapshot-005000.pkl


In [ ]:
network_pkl_list = []
network_pkl_list += [
                     '/content/' + model_name
                     for model_name in sorted(model_names)
                     ]

print('\n'.join(network_pkl_list))                     

/content/network-snapshot-001014.pkl
/content/network-snapshot-002003.pkl
/content/network-snapshot-003001.pkl
/content/network-snapshot-004006.pkl
/content/network-snapshot-005000.pkl


In [ ]:
network_pkl_list = []
network_pkl_list += [
                     '/content/' + model_name
                     for model_name in sorted(ffhq_model_names)
                     ]

print('\n'.join(network_pkl_list))

/content/stylegan2-ffhq-config-e.pkl
/content/stylegan2-ffhq-config-f.pkl


In [ ]:
%cd /content/stylegan2/

/content/stylegan2


In [ ]:
import dnnlib
import dnnlib.tflib as tflib

Gs_syn_kwargs = dnnlib.EasyDict()
Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8,
                                      nchw_to_nhwc=True)
Gs_syn_kwargs.randomize_noise = False
Gs_syn_kwargs.minibatch_size = 4

In [ ]:
output_folder_for_average_images = 'average_image/'
%mkdir -p $output_folder_for_average_images

In [ ]:
file_extension = '.jpg'

In [ ]:
import pretrained_networks
import numpy as np
import PIL.Image
from pathlib import Path

n = len(network_pkl_list)
size = 256
canvas = PIL.Image.new('RGB', (n * size, size))
for (i, network_pkl)  in enumerate(network_pkl_list):  
    print(network_pkl)

    _, _, Gs = pretrained_networks.load_networks(network_pkl)
    
    w_avg = Gs.get_var('dlatent_avg')

    # Reference: https://github.com/NVlabs/stylegan2/blob/cec605e0834de5404d5c7e5cead7053bdd0e4dde/projector.py#L95
    w = np.tile(w_avg, [1, Gs.components.synthesis.input_shape[1], 1])

    image = Gs.components.synthesis.run(w, **Gs_syn_kwargs)[0]
    image = PIL.Image.fromarray(image)

    output_root_file_name = Path(network_pkl).name.replace('.pkl', '')

    image.save('{}{}{}'.format(output_folder_for_average_images,
                             output_root_file_name,
                             file_extension))

    image = image.resize((size, size), PIL.Image.LANCZOS)
    canvas.paste(image, (i * size, 0))

canvas.save('{}average_image{}'.format(output_folder_for_average_images,
                                       file_extension))

/content/stylegan2-ffhq-config-e.pkl
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.
/content/stylegan2-ffhq-config-f.pkl


In [ ]:
!tar -cf average_image.tar.gz average_image/

## GANspace

[2] [Härkönen, Erik, et al. *GANSpace: Discovering Interpretable GAN Controls*. arXiv 2020.][ganspace-paper] **([code][ganspace])**

[ganspace-paper]: <https://arxiv.org/abs/2004.02546>
[ganspace]: <https://github.com/harskish/ganspace>

My fork: https://github.com/woctezuma/ganspace/tree/steam

The original GANSpace notebook: [Github][Ganspace_original-github] and [Colab][Ganspace_original-colab]

A Colab notebook (very similar to the original GANSpace notebook above): [Colab][Ganspace_similar-colab]

My version of the GANSpace notebook (adapted to use my fork): [Github][Ganspace_colab_for_steam-github] and [Colab][Ganspace_colab_for_steam-colab]

[Ganspace_original-github]: <https://github.com/harskish/ganspace/blob/master/notebooks/Ganspace_colab.ipynb>
[Ganspace_original-colab]: <https://colab.research.google.com/github/harskish/ganspace/blob/master/notebooks/Ganspace_colab.ipynb>
[Ganspace_similar-colab]: <https://colab.research.google.com/drive/1g-ShMzkRWDMHPyjom_p-5kqkn2f-GwBi>
[Ganspace_colab_for_steam-github]: <https://github.com/woctezuma/steam-stylegan2/blob/master/StyleGAN2_latent_discovery.ipynb>
[Ganspace_colab_for_steam-colab]: <https://colab.research.google.com/github/woctezuma/steam-stylegan2/blob/ganspace/Ganspace_colab_for_steam.ipynb>

## Closed-Form Factorization

[3] [Shen, Yujun, et al. *Closed-Form Factorization of Latent Semantics in GANs*. arXiv 2020.][closed-form-paper] **([code][closed-form])**

[closed-form-paper]: <https://arxiv.org/abs/2007.06600>
[closed-form]: <https://github.com/rosinality/stylegan2-pytorch#closed-form-factorization-httpsarxivorgabs200706600>

In [ ]:
%cd /content/stylegan2-pytorch/

/content/stylegan2-pytorch


### Extract eigenvectors of the weight matrix

#### Run

In [ ]:
!python closed_form_factorization.py $pytorch_model_name

#### Check

In [ ]:
import torch

eigvec = torch.load('factor.pt')["eigvec"].to('cuda')

In [ ]:
eigvec.shape

torch.Size([512, 512])

### Test the meaning of extracted directions

In [ ]:
degree = 5
num_degrees = 5

In [ ]:
truncation = 0.75

assert(0 <= truncation <= 1)

In [ ]:
output_keyword = 'temp_degree_{:03d}_truncation_{}'.format(degree, 
                                                           truncation)

print(output_keyword)

temp_degree_005_truncation_0.75


In [ ]:
output_folder = output_keyword + '/'
out_prefix = output_folder + 'factor'

In [ ]:
%rm -rf $output_folder
%mkdir -p $output_folder

##### Run

In [ ]:
print('PyTorch model name: {}'.format(pytorch_model_name))
print('Files will be saved to {}'.format(output_folder))

PyTorch model name: stylegan2-ffhq-config-e.pt
Files will be saved to temp_degree_005_truncation_0.75/


In [ ]:
if debug_with_ffhq:
  image_size = 1024
else:
  image_size = 256

In [ ]:
num_eigenvectors = 100
num_samples = 10

In [ ]:
for eigenvector_index in range(num_eigenvectors):

  print('Eigenvector n°{:02d}/{:02d}'.format(eigenvector_index+1, 
                                             num_eigenvectors))

  !python apply_factor.py \
    --index $eigenvector_index \
    --degree $degree \
    --degree_num $num_degrees \
    --channel_multiplier 1 \
    --torch_seed 2021 \
    --ckpt $pytorch_model_name \
    --size $image_size \
    --n_sample $num_samples \
    --truncation $truncation \
    --out_prefix $out_prefix \
    factor.pt

#### Archive

In [ ]:
archive_name = output_keyword + '.tar.gz'

In [ ]:
!tar -cf $archive_name $output_folder
!du -sh $archive_name

527M	temp_degree_005_truncation_0.75.tar.gz
